# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [6]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [7]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [8]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token('pk.eyJ1IjoiZ3IzbnozbiIsImEiOiJjbDE3eHF4bW4wbHBvM2pvMzd0NmdrMmk0In0.V4RLZgz5bKsUFmRWrBbGyQ')

# Import Data

In [9]:
# Import the CSVs to Pandas DataFrames

In [58]:
to_data = pd.read_csv("toronto_neighbourhoods_census_data.csv", index_col="year")
df_neighbourhood_locations=pd.read_csv("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations

,neighbourhood,lat,lon
0,Agincourt North,43.805441,-79.266712
1,Agincourt South-Malvern West,43.788658,-79.265612
2,Alderwood,43.604937,-79.541611
3,Annex,43.671585,-79.404001
4,Banbury-Don Mills,43.737657,-79.349718
...,...,...,...
135,Wychwood,43.676919,-79.425515
136,Yonge-Eglinton,43.704689,-79.403590
137,Yonge-St.Clair,43.687859,-79.397871
138,York University Heights,43.765736,-79.488883


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [59]:
# Getting the data from the top 10 expensive neighbourhoods
neighbourhood_avg_value=to_data.reset_index()
dwelling=neighbourhood_avg_value.groupby('neighbourhood').mean()
top_ten=dwelling.sort_values('average_house_value',ascending=False)
top_ten.drop(columns=['year'],inplace=True)


In [17]:
# Calculate the mean number of dwelling types units per year
house_types=to_data.drop(columns=['neighbourhood','shelter_costs_owned','shelter_costs_rented'])
house_types=house_types.groupby('year').mean()
house_types

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value
year,,,,,,,,,
2001,2149.500000,2535.821429,0.535714,649.964286,373.964286,169.892857,835.000000,21.714286,289882.885714
2006,1906.142857,2710.000000,1.178571,495.928571,390.642857,314.964286,1163.214286,9.535714,424059.664286
2011,1963.857143,3065.857143,0.714286,517.714286,431.107143,319.642857,1170.678571,15.464286,530424.721429
2016,1926.285714,3523.357143,0.678571,508.571429,439.750000,347.035714,1182.678571,20.321429,664068.328571


In [16]:
# Calculate the average monthly shelter costs for owned and rented dwellings
avg_shelter_cost=to_data[['shelter_costs_owned','shelter_costs_rented']].groupby(['year']).mean()
avg_shelter_cost

,shelter_costs_owned,shelter_costs_rented
year,,
2001,846.878571,1085.935714
2006,1316.800000,925.414286
2011,1448.214286,1019.792857
2016,1761.314286,1256.321429


### Panel Visualization Functions

In [53]:
loc_value=()
# Define Panel visualization functions
def neighbourhood_map():
    neighborhood_plot = px.scatter_mapbox(
        loc_value,
        lat='lat',
        lon='lon',
        size='average_house_value',
        color='shelter_costs_owned',
        color_continuous_scale=px.colors.cyclical.IceFire,
        title='Toronto Average House Values',
        zoom=5,
        width=850,
        height=500
    )
    return neighborhood_plot

In [27]:
def create_bar_chart(data, title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    bar_chart=data.drop(columns=['average_house_value','shelter_costs_owned','shelter_costs_rented'])
    bar_chart=bar_chart.transpose()
    bar_plot_chart=bar_chart[[y]].plot(kind='bar',title=title,color=color,rot=90)
    bar_plot=pn.panel(bar_plot_chart.figure)
    return bar_plot
    

In [28]:
def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    line_chart=data.plot(kind='line',title=title,xlabel=xlabel, ylabel=ylabel,color=color)
    plot_line_chart=pn.panel(plot_line_chart.figure)
    return plot_line_chart

In [29]:

def average_house_value():
    """Average house values per year."""
    
    average_house_value=house_types.hvplot.line(title='Toronto Average Home Value',
                                                xlabel='Average Home Value',
                                                ylabel='Year')
    return average_house_value

In [30]:

def average_value_by_neighbourhood():
    """Average house values by neighbourhood."""
    neighbourhood_plot=data.hvplot.line(title='Average neighbourhood house value', rot=90, groupby=groupby)
    neighbourhood_plot

In [31]:

def number_dwelling_types():
    """Number of dwelling types per year"""
    dwelling_types_plot=data.hvplot.bar(title='Dwelling types per neighbourhood',ylabel='Dwelling Types',xlabel='Year',rot=90, groupby=groupby,frame_width=500,frame_height=500) 
    return dwelling_types_plot

In [32]:

def average_house_value_snapshot():
    """Average house value for all Toronto's neighbourhoods per year."""
    snapshot=px.bar(data,title='Average House Values in Toronto per Neighbourhood',x=data[x],y=data[y],color=color,facet_row=facet_row,height=850)
    snapshot_panel=pn.panel(snapshot)
    
    return snapshot_panel

In [ ]:

def top_most_expensive_neighbourhoods():
    """Top 10 most expensive neighbourhoods."""
    most_expencive=data.iloc[0:ranking]
    data_neighbourhoods=data['average_house_value'].hvplot.bar(title=f'Top {ranking} expensive neighbourhoods',ylabel='Average house value',xlabel='Neighbourhoods',rot=90,frame_width=500,frame_height=500)
    top_neighbourhoods_plot = pn.panel(data_neighbourhoods)
    
    return top_neighbourhoods_plot


In [33]:

def sunburts_cost_analysis():
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    sunburst_plot=px.sunburst(data,title='Home costs analysis per year in Torontos Most Expensive Neighbourhoods',path=path,values=values,color=color)
    cost_sunburst_plot = pn.panel(sunburst_plot)
    
    return cost_sunburst_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [54]:
# Create a Title for the Dashboard
dash_title = '# Real Estate Analysis 2001 - 2016 for the city of Toronto'

# Define a welcome text
def welcome_text():
        welcome = 'Welcome and enjoy this Visual analysis of Toronto Census data betwenn 2001 and 2016'
        return welcome

# Create a tab layout for the dashboard
# Create the main dashboard
welcome_tab=pn.Column(welcome_text(),neighbourhood_map())

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of [] but received: average_house_value

In [55]:
yr_mkt_analysis=pn.Column(pn.Row(create_bar_chart(house_types,'Dwelling Types in Toronto in 2001',2001,'Dwelling Types','Dwelling Types Units','green'),
                                   create_bar_chart(house_types,'Dwelling Types in Toronto in 2006',2006,'Dwelling Types','Dwelling Types Units','red'),sizing_mode='stretch_height'),
                            pn.Row(create_bar_chart(house_types,'Dwelling Types in Toronto in 2011',2011,'Dwelling Types','Dwelling Types Units','yellow'),
                                   create_bar_chart(house_types,'Dwelling Types in Toronto in 2016',2016,'Dwelling Types','Dwelling Types Units','navy'),sizing_mode='stretch_height'),sizing_mode='stretch_height')

TypeError: create_bar_chart() takes 5 positional arguments but 6 were given

In [41]:
sh_cost_v_home_val=pn.Column(create_line_chart(avg_shelter_cost[['shelter_costs_owned']],'Average Montly Costs for Owned Dwellings in Toronto','Year','Avg Monthly Shelter Costs','blue'),
                    create_line_chart(avg_shelter_cost[['shelter_costs_rented']],'Average Montly Costs for Rented Dwellings in Toronto','Year','Avg Monthly Shelter Costs','yellow'),
                    average_house_value())

neigh_analysis=pn.Row(pn.Column(average_value_by_neighbourhood(mean_house_prices,'neighbourhood'),
             number_dwelling_types(num_dwelling,'neighbourhood','year')),
             snapshot_panel(num_dwelling,'neighbourhood','average_house_value','average_house_value','year'))

top_exp_neigh=pn.Row(top_most_expensive_neighbourhoods(sorted_dwelling,10),
                       sunburts_cost_analysis(ranked_dwelling_all,['year','neighbourhood'],'average_house_value','shelter_costs_owned'))

tab=pn.Tabs(('Welcome',welcome_tab),
            ('Yearly Market',yr_mkt_analysis),
            ('Shelter Costs Vs. House Value',sh_cost_v_home_val),
            ('Neighbourhood Analysis',neigh_analysis),
            ('Top Expensive Neighbourhoods',top_exp_neigh))
dashboard = pn.Column(dash_title,tab);

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of [] but received: average_house_value

## Serve the Panel Dashboard

In [ ]:
dashboard.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [56]:
# neighbourhood_map().show()
neighbourhood_map().show()

ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of [] but received: average_house_value

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()